In [123]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import lit, current_timestamp, regexp_replace, from_json, col, round as spark_round, when
from dotenv import load_dotenv
import requests, os, unicodedata
from datetime import datetime
# Cargar variables de entorno
load_dotenv("/home/jovyan/work/.env")

# Variables necesarias
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT")
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_DATABASE = os.getenv("MYSQL_DATABASE")

# Crear sesión de Spark
spark = SparkSession.builder \
    .appName("Fase 2 - Limpieza") \
    .config("spark.jars", "/home/jovyan/work/jars/mysql-connector-j-8.0.31.jar") \
    .getOrCreate()

In [124]:
def obtener_id_carga(origen: str, archivo: str) -> int:
    log_df = spark.read \
        .format("jdbc") \
        .option("url", f"jdbc:mysql://{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}") \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .option("dbtable", "log_carga") \
        .option("user", MYSQL_USER) \
        .option("password", MYSQL_PASSWORD) \
        .load()
    
    result = log_df \
        .filter((log_df.origen == origen) & (log_df.archivo == archivo)) \
        .orderBy(log_df.id.desc()) \
        .limit(1) \
        .collect()
    
    if result:
        return result[0]["id"]
    else:
        raise ValueError(f"No se encontró entrada en log_carga para {origen} - {archivo}")

def insertar_log_limpieza(origen, archivo, cantidad, inicio, fin, detalles=""):
    log_df = spark.createDataFrame([{
        "origen": origen,
        "archivo": archivo,
        "cantidad": int(cantidad),
        "fecha_inicio": inicio,
        "fecha_fin": fin,
        "detalles": detalles
    }])
    
    log_df.write \
        .format("jdbc") \
        .option("url", f"jdbc:mysql://{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}") \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .option("dbtable", "log_limpieza") \
        .option("user", MYSQL_USER) \
        .option("password", MYSQL_PASSWORD) \
        .mode("append") \
        .save()

**CARGA, LIMPIEZA Y PROCESADO DE TWITCH API**

In [125]:
# Leer los datos crudos desde Parquet
df_twitch = spark.read.parquet("/home/jovyan/work/twitch_top_games.parquet")

# Mostrar esquema y primeras filas
df_twitch.printSchema()
df_twitch.show(5)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- box_art_url: string (nullable = true)
 |-- igdb_id: string (nullable = true)
 |-- total_viewers: long (nullable = true)
 |-- num_streams: long (nullable = true)
 |-- avg_viewers: double (nullable = true)
 |-- languages: string (nullable = true)
 |-- has_mature_stream: boolean (nullable = true)

+----------+--------------------+--------------------+-------+-------------+-----------+-----------------+--------------------+-----------------+
|        id|                name|         box_art_url|igdb_id|total_viewers|num_streams|      avg_viewers|           languages|has_mature_stream|
+----------+--------------------+--------------------+-------+-------------+-----------+-----------------+--------------------+-----------------+
|      2748|Magic: The Gathering|https://static-cd...|       |         3984|         20|            199.2|{"en": 15, "pt": ...|             true|
|2068583461|            NBA 2K25|https:/

In [126]:
from pyspark.sql.functions import col, lit, current_timestamp, regexp_replace, round as spark_round, when
import unicodedata

inicio_limpieza = datetime.now()
#Eliminación de duplicados
df_twitch = df_twitch.dropDuplicates()

#Eliminar nulos
df_twitch = df_twitch.na.drop(how="all")

#Normalizar nombres de columnas

def normalizar_col(col_name):
    col_name = unicodedata.normalize('NFKD', col_name).encode('ASCII', 'ignore').decode('utf-8')
    return col_name.lower().replace(" ", "_")

df_twitch = df_twitch.toDF(*[normalizar_col(c) for c in df_twitch.columns])

In [127]:
dw_id = obtener_id_carga("Twitch", "twitch_top_games.parquet")

df_twitch = df_twitch.withColumn(
    "box_art_url",
    regexp_replace("box_art_url", r"\{width\}", "285")
)

df_twitch = df_twitch.withColumn(
    "box_art_url",
    regexp_replace("box_art_url", r"\{height\}", "380")
)

df_twitch = df_twitch.withColumn("avg_viewers", spark_round(col("avg_viewers"), 2))
df_twitch = df_twitch.withColumn("igdb_id", when(col("igdb_id").isNull() | (col("igdb_id") == ""), lit("0")).otherwise(col("igdb_id")))
df_twitch = df_twitch.withColumn("has_mature_stream", when(col("has_mature_stream") == True, lit(1)).otherwise(lit(0)))
df_twitch = df_twitch.withColumn("languages", when(col("languages").isNull(), lit("{}")).otherwise(col("languages")))

df_twitch = df_twitch \
    .withColumn("dw_fecha_registro", current_timestamp()) \
    .withColumn("dw_id_carga", lit(dw_id)) \
    .withColumn("dw_deleted", lit(0)) \
    .withColumn("dw_source", lit("Twitch")) \
    .withColumn("dw_endpoint", lit("https://api.twitch.tv/helix/games/top + /streams"))



DataFrame[id: string, name: string, box_art_url: string, igdb_id: string, total_viewers: bigint, num_streams: bigint, avg_viewers: double, languages: string, has_mature_stream: int, dw_fecha_registro: timestamp, dw_id_carga: int, dw_deleted: int, dw_source: string, dw_endpoint: string]

In [128]:
df_twitch.write \
    .format("jdbc") \
    .option("url", f"jdbc:mysql://{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "core_twitch_games") \
    .option("user", MYSQL_USER) \
    .option("password", MYSQL_PASSWORD) \
    .mode("overwrite") \
    .save()

In [129]:
fin_limpieza = datetime.now()

insertar_log_limpieza(
    origen="Twitch",
    archivo="core_twitch_games",
    cantidad=df_twitch.count(),
    inicio=inicio_limpieza,
    fin=fin_limpieza,
    detalles="Limpieza básica aplicada: duplicados, nulos, normalización, columnas técnicas"
)

**CARGA, LIMPIEZA Y PROCESADO DE HOWLONGTOBEAT**


In [111]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, BooleanType, ArrayType

# Esquema del campo anidado json_content
json_content_schema = StructType([
    StructField("game_id", IntegerType()),
    StructField("game_name", StringType()),
    StructField("game_name_date", IntegerType()),
    StructField("game_alias", StringType()),
    StructField("game_type", StringType()),
    StructField("game_image", StringType()),
    StructField("comp_lvl_combine", IntegerType()),
    StructField("comp_lvl_sp", IntegerType()),
    StructField("comp_lvl_co", IntegerType()),
    StructField("comp_lvl_mp", IntegerType()),
    StructField("comp_main", IntegerType()),
    StructField("comp_plus", IntegerType()),
    StructField("comp_100", IntegerType()),
    StructField("comp_all", IntegerType()),
    StructField("comp_main_count", IntegerType()),
    StructField("comp_plus_count", IntegerType()),
    StructField("comp_100_count", IntegerType()),
    StructField("comp_all_count", IntegerType()),
    StructField("invested_co", IntegerType()),
    StructField("invested_mp", IntegerType()),
    StructField("invested_co_count", IntegerType()),
    StructField("invested_mp_count", IntegerType()),
    StructField("count_comp", IntegerType()),
    StructField("count_speedrun", IntegerType()),
    StructField("count_backlog", IntegerType()),
    StructField("count_review", IntegerType()),
    StructField("review_score", IntegerType()),
    StructField("count_playing", IntegerType()),
    StructField("count_retired", IntegerType()),
    StructField("profile_platform", StringType()),
    StructField("profile_popular", IntegerType()),
    StructField("release_world", IntegerType())
])

# Esquema completo del raw_json
raw_json_schema = StructType([
    StructField("game_id", IntegerType()),
    StructField("game_name", StringType()),
    StructField("game_alias", StringType()),
    StructField("game_type", StringType()),
    StructField("game_image_url", StringType()),
    StructField("game_web_link", StringType()),
    StructField("review_score", IntegerType()),
    StructField("profile_dev", StringType()),
    StructField("profile_platforms", ArrayType(StringType())),
    StructField("release_world", IntegerType()),
    StructField("similarity", FloatType()),
    StructField("json_content", json_content_schema),
    StructField("main_story", FloatType()),
    StructField("main_extra", FloatType()),
    StructField("completionist", FloatType()),
    StructField("all_styles", FloatType()),
    StructField("coop_time", FloatType()),
    StructField("mp_time", FloatType()),
    StructField("complexity_lvl_combine", BooleanType()),
    StructField("complexity_lvl_sp", BooleanType()),
    StructField("complexity_lvl_co", BooleanType()),
    StructField("complexity_lvl_mp", BooleanType())
])

In [112]:
from pyspark.sql.functions import from_json, col, lit, current_timestamp

# Leer parquet crudo
df_crudo = spark.read.parquet("/home/jovyan/work/howlongtobeat_multiple_raw.parquet")

inicio_limpieza = datetime.now()
# Parsear raw_json
df_limpio = df_crudo.withColumn("json_data", from_json(col("raw_json"), raw_json_schema)).select("json_data.*")

# Limpieza básica
df_limpio = df_limpio.dropDuplicates().na.drop(how="all")

# Separar plataformas
df_limpio = df_limpio \
    .withColumn("platform_1", col("profile_platforms").getItem(0)) \
    .withColumn("platform_2", col("profile_platforms").getItem(1)) \
    .withColumn("platform_3", col("profile_platforms").getItem(2)) \
    .withColumn("platform_4", col("profile_platforms").getItem(3)) \
    .withColumn("platform_5", col("profile_platforms").getItem(4))

# Añadir columnas técnicas
dw_id = obtener_id_carga("HowLongToBeat", "howlongtobeat_multiple_raw.parquet")

df_hltb_final = df_limpio \
    .withColumn("dw_fecha_registro", current_timestamp()) \
    .withColumn("dw_id_carga", lit(dw_id)) \
    .withColumn("dw_deleted", lit(0).cast("int")) \
    .withColumn("dw_source", lit("HowLongToBeat")) \
    .withColumn("dw_endpoint", lit("https://www.howlongtobeat.com/api"))

df_hltb_final = df_hltb_final.drop("json_content")
df_hltb_final.limit(10).toPandas()

,game_id,game_name,game_alias,game_type,game_image_url,game_web_link,review_score,profile_dev,profile_platforms,release_world,...,platform_1,platform_2,platform_3,platform_4,platform_5,dw_fecha_registro,dw_id_carga,dw_deleted,dw_source,dw_endpoint
0,36732,Pac-Man 256,,game,https://howlongtobeat.com/games/36732_Pac-Man_...,https://howlongtobeat.com/game/36732,69,None,"[Mobile, PC, PlayStation 4, Xbox One]",2016,...,Mobile,PC,PlayStation 4,Xbox One,None,2025-05-27 11:28:25.521620,2,0,HowLongToBeat,https://www.howlongtobeat.com/api
1,11375,X-Men vs. Street Fighter,,game,https://howlongtobeat.com/games/11375_X-Men_vs...,https://howlongtobeat.com/game/11375,76,None,"[Arcade, PlayStation, Sega Saturn]",1996,...,Arcade,PlayStation,Sega Saturn,None,None,2025-05-27 11:28:25.521620,2,0,HowLongToBeat,https://www.howlongtobeat.com/api
2,19348,Ultra Street Fighter IV,Ultra Street Fighter 4,game,https://howlongtobeat.com/games/ultra_street_f...,https://howlongtobeat.com/game/19348,79,None,"[Arcade, PC, PlayStation 3, PlayStation 4, Xbo...",2014,...,Arcade,PC,PlayStation 3,PlayStation 4,Xbox 360,2025-05-27 11:28:25.521620,2,0,HowLongToBeat,https://www.howlongtobeat.com/api
3,24372,Tetris Ultimate,,game,https://howlongtobeat.com/games/Tetris_Ultimat...,https://howlongtobeat.com/game/24372,69,None,"[Nintendo 3DS, PlayStation 4, PlayStation Vita...",2014,...,Nintendo 3DS,PlayStation 4,PlayStation Vita,Xbox One,None,2025-05-27 11:28:25.521620,2,0,HowLongToBeat,https://www.howlongtobeat.com/api
4,64989,Tetris 99,,multi,https://howlongtobeat.com/games/64989_Tetris_9...,https://howlongtobeat.com/game/64989,78,None,[Nintendo Switch],2019,...,Nintendo Switch,None,None,None,None,2025-05-27 11:28:25.521620,2,0,HowLongToBeat,https://www.howlongtobeat.com/api
5,26433,Pac-Man & Galaga Dimensions,Pac-Man and Galaga Dimensions,game,https://howlongtobeat.com/games/26433_Pac-Man_...,https://howlongtobeat.com/game/26433,45,None,[Nintendo 3DS],2011,...,Nintendo 3DS,None,None,None,None,2025-05-27 11:28:25.521620,2,0,HowLongToBeat,https://www.howlongtobeat.com/api
6,108883,Street Fighter 6,Street Fighter VI,game,https://howlongtobeat.com/games/108883_Street_...,https://howlongtobeat.com/game/108883,83,None,"[Nintendo Switch 2, PC, PlayStation 4, PlaySta...",2023,...,Nintendo Switch 2,PC,PlayStation 4,PlayStation 5,Xbox Series X/S,2025-05-27 11:28:25.521620,2,0,HowLongToBeat,https://www.howlongtobeat.com/api
7,9243,Street Fighter III: New Generation,Street Fighter 3: New Generation,game,https://howlongtobeat.com/games/Street_Fighter...,https://howlongtobeat.com/game/9243,72,None,"[Arcade, Dreamcast, Nintendo Switch, PC, PlayS...",1997,...,Arcade,Dreamcast,Nintendo Switch,PC,PlayStation 4,2025-05-27 11:28:25.521620,2,0,HowLongToBeat,https://www.howlongtobeat.com/api
8,19930,Pac-Man Fever,,game,https://howlongtobeat.com/games/Pac-Man_Fever_...,https://howlongtobeat.com/game/19930,47,None,"[Nintendo GameCube, PlayStation 2]",2002,...,Nintendo GameCube,PlayStation 2,None,None,None,2025-05-27 11:28:25.521620,2,0,HowLongToBeat,https://www.howlongtobeat.com/api
9,9719,Tetris DS,,game,https://howlongtobeat.com/games/200px-Tetris_D...,https://howlongtobeat.com/game/9719,81,None,"[Nintendo 3DS, Nintendo DS]",2006,...,Nintendo 3DS,Nintendo DS,None,None,None,2025-05-27 11:28:25.521620,2,0,HowLongToBeat,https://www.howlongtobeat.com/api


In [113]:
# Guardar en MySQL en la tabla core_howlongtobeat
df_hltb_final = df_hltb_final.drop("profile_platforms")

df_hltb_final.write \
    .format("jdbc") \
    .option("url", f"jdbc:mysql://{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "core_howlongtobeat") \
    .option("user", MYSQL_USER) \
    .option("password", MYSQL_PASSWORD) \
    .mode("overwrite") \
    .save()

In [114]:
# Guardar en MySQL en la tabla log_limpieza

cantidad = df_hltb_final.count()
fin_limpieza = datetime.now()

insertar_log_limpieza(
    origen="HowLongToBeat",
    archivo="core_howlongtobeat",
    cantidad=cantidad,
    inicio=inicio_limpieza,
    fin=fin_limpieza,
    detalles="Limpieza completa: duplicados, nulos, json_content, plataformas, columnas técnicas"
)

**CARGA, LIMPIEZA Y PROCESADO DE POKEMON SHOWDOWN POKEDEX**

In [130]:
df_showdown_raw = spark.read.parquet("/home/jovyan/work/pokemon_showdown_pokedex_raw.parquet")

raw_str = df_showdown_raw.collect()[0]["raw_json"]

dw_id = obtener_id_carga("Pokemon Showdown", "pokemon_showdown_pokedex_raw.parquet")

In [131]:
#Inicio de la limpieza de los datos
inicio_limpieza = datetime.now()

# Eliminar llaves externas
contenido = raw_str[raw_str.find("{") + 1 : raw_str.rfind("}")]
contenido = contenido.replace('\t', '').replace('\n', '')

# Dividir en bloques por Pokémon
bloques = re.findall(r'(\w+):\s*\{((?:[^{}]|\{[^{}]*\})*?)\},?', contenido)
pokemons = []

def extraer_valor(campo, texto):
    patron = fr'"?{campo}"?\s*:\s*("[^"]*"|\[[^\]]*\]|\{{[^}}]*\}}|[^,]+)'
    encontrado = re.search(patron, texto)
    if not encontrado:
        return None
    valor = encontrado.group(1).strip()
    # Si es un dict, añadir comillas a claves internas
    if valor.startswith("{") and ":" in valor:
        valor = re.sub(r'([,{]\s*)([a-zA-Z0-9_]+):', r'\1"\2":', valor)
        
    return eval(valor)

for nombre, cuerpo in bloques:
    pokemon = {"id": nombre}
    pokemon["name"] = extraer_valor("name", cuerpo)
    pokemon["num"] = extraer_valor("num", cuerpo)
    pokemon["heightm"] = extraer_valor("heightm", cuerpo)
    pokemon["weightkg"] = extraer_valor("weightkg", cuerpo)
    pokemon["color"] = extraer_valor("color", cuerpo)
    pokemon["prevo"] = extraer_valor("prevo", cuerpo)
    pokemon["evoType"] = extraer_valor("evoType", cuerpo)
    pokemon["canGigantamax"] = extraer_valor("canGigantamax", cuerpo)

    tipos = extraer_valor("types", cuerpo) or []
    for i, tipo in enumerate(tipos):
        pokemon[f"type_{i+1}"] = tipo

    stats = extraer_valor("baseStats", cuerpo) or {}
    for k, v in stats.items():
        pokemon[k] = v

    abilities = extraer_valor("abilities", cuerpo) or {}
    for k, v in abilities.items():
        clave = "ability_" + ("H" if k == "H" else str(k))
        pokemon[clave] = v

    evos = extraer_valor("evos", cuerpo) or []
    for i, evo in enumerate(evos):
        pokemon[f"evo_{i+1}"] = evo

    pokemons.append(pokemon)

# Convertir a DataFrame
df_pokedex = pd.DataFrame(pokemons)

# Copiamos el dataframe original por seguridad
df_clean = df_pokedex.copy()

# Consolidar evoluciones: tomamos solo la primera (si hay)
df_clean["evo"] = df_clean[[col for col in df_clean.columns if col.startswith("evo_")]].bfill(axis=1).iloc[:, 0]
df_clean.drop(columns=[col for col in df_clean.columns if col.startswith("evo_")], inplace=True)

# Filtrar columnas de habilidades deseadas
columns_to_keep = ["ability_0", "ability_1", "ability_H"]
ability_cols = [c for c in df_clean.columns if c.startswith("ability_")]
drop_ability = [c for c in ability_cols if c not in columns_to_keep]
df_clean.drop(columns=drop_ability, inplace=True)

# Orden final de columnas sugeridas
column_order = [
    "id", "name", "num", "heightm", "weightkg", "color", "prevo", "evo", "evoType", "canGigantamax",
    "type_1", "type_2", "hp", "atk", "def", "spa", "spd", "spe", "ability_0", "ability_1", "ability_H"
]

# Reordenar si todas están presentes
df_clean = df_clean[[c for c in column_order if c in df_clean.columns]]

# Verifica que está bien
df_clean.head()

,id,name,num,heightm,weightkg,color,prevo,evo,evoType,canGigantamax,...,type_2,hp,atk,def,spa,spd,spe,ability_0,ability_1,ability_H
0,bulbasaur,Bulbasaur,1,0.7,6.9,Green,None,Ivysaur,None,None,...,Poison,45,49,49,65,65,45,Overgrow,NaN,Chlorophyll
1,ivysaur,Ivysaur,2,1.0,13.0,Green,Bulbasaur,Venusaur,None,None,...,Poison,60,62,63,80,80,60,Overgrow,NaN,Chlorophyll
2,venusaur,Venusaur,3,2.0,100.0,Green,Ivysaur,NaN,None,G-Max Vine Lash,...,Poison,80,82,83,100,100,80,Overgrow,NaN,Chlorophyll
3,venusaurmega,Venusaur-Mega,3,2.4,155.5,Green,None,NaN,None,None,...,Poison,80,100,123,122,120,80,Thick Fat,NaN,NaN
4,venusaurgmax,Venusaur-Gmax,3,24.0,0.0,Green,None,NaN,None,None,...,Poison,80,82,83,100,100,80,Overgrow,NaN,Chlorophyll


In [133]:
#Dar sentido a los nulos y a las formas especiales de combate

# A: Detectar formas especiales (gmax, mega, totem)
df_clean["is_special_form"] = df_clean["id"].str.contains("gmax|mega|totem", case=False)

# B: Reasignar prevo, evo y evoType para formas especiales
df_clean.loc[df_clean["is_special_form"], "prevo"] = "base"
df_clean.loc[df_clean["is_special_form"], "evo"] = "last_form"
df_clean.loc[df_clean["is_special_form"], "evoType"] = "form_only"

#C: Limpiar nulos
df_clean["evoType"] = df_clean["evoType"].fillna("level")
df_clean["prevo"] = df_clean["prevo"].fillna("base")
df_clean["evo"] = df_clean["evo"].fillna("last evolution")
df_clean["type_2"] = df_clean["type_2"].fillna("monotype")
df_clean["ability_1"] = df_clean["ability_1"].fillna("No ability 1")
df_clean["ability_H"] = df_clean["ability_H"].fillna("No hidden ability")
df_clean["canGigantamax"] = df_clean["canGigantamax"].fillna("No Gigantamax")

# Verifica que está bien
df_clean.head()

#convertir a dataframe de spark
df_clean_spark = spark.createDataFrame(df_clean)

df_pokedex_final = df_clean_spark \
    .withColumn("dw_fecha_registro", current_timestamp()) \
    .withColumn("dw_id_carga", lit(dw_id)) \
    .withColumn("dw_deleted", lit(0).cast("int")) \
    .withColumn("dw_source", lit("Pokemon Showdown")) \
    .withColumn("dw_endpoint", lit("https://github.com/smogon/pokemon-showdown/blob/master/data/pokedex.ts"))

In [118]:
#GUARDAR en MySQL en la tabla de core_pokedex

df_pokedex_final.write \
    .format("jdbc") \
    .option("url", f"jdbc:mysql://{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "core_pokedex") \
    .option("user", MYSQL_USER) \
    .option("password", MYSQL_PASSWORD) \
    .mode("overwrite") \
    .save()

In [119]:
# Guardar en MySQL en la tabla log_limpieza

cantidad = df_pokedex_final.count()
fin_limpieza = datetime.now()

insertar_log_limpieza(
    origen="Pokemon Showdown Pokedex",
    archivo="core_pokemon",
    cantidad=cantidad,
    inicio=inicio_limpieza,
    fin=fin_limpieza,
    detalles="Limpieza completa: división por Pokémon, extracción de stats, tipos y habilidades, relleno de campos nulos, formas especiales Gmax/Mega manejadas"
)

In [120]:
# pd.set_option("display.max_rows", None)      # Mostrar todas las filas
# pd.set_option("display.max_columns", None)   # Mostrar todas las columnas
# pd.set_option("display.width", None)         # Ajustar el ancho al contenido
# pd.set_option("display.max_colwidth", None)  # Mostrar columnas largas

# df_clean

In [121]:
#TABLAS EN MySQL

# CREATE TABLE log_limpieza (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     origen VARCHAR(50),
#     archivo VARCHAR(255),
#     cantidad INT,
#     fecha_inicio DATETIME,
#     fecha_fin DATETIME,
#     detalles TEXT
# ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

# CREATE TABLE core_twitch_games (
#     id BIGINT,
#     name VARCHAR(255),
#     box_art_url TEXT,
#     igdb_id BIGINT,
#     dw_fecha_registro DATETIME,
#     dw_id_carga INT,
#     dw_deleted TINYINT(1) DEFAULT 0 NOT NULL,
#     dw_source VARCHAR(50),
#     dw_endpoint TEXT
# ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

# CREATE TABLE core_howlongtobeat (
#     game_id BIGINT,
#     game_name VARCHAR(255),
#     review_score INT,
#     release_year INT,
#     main_story FLOAT,
#     main_extra FLOAT,
#     completionist FLOAT,
#     all_styles FLOAT,
#     coop_time FLOAT,
#     mp_time FLOAT,
#     dw_fecha_registro DATETIME,
#     dw_id_carga INT,
#     dw_deleted TINYINT(1) DEFAULT 0 NOT NULL,
#     dw_source VARCHAR(50),
#     dw_endpoint TEXT
# ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

# CREATE TABLE core_pokedex (
#     id VARCHAR(50) PRIMARY KEY,
#     name VARCHAR(100),
#     num INT,
#     heightm FLOAT,
#     weightkg FLOAT,
#     color VARCHAR(30),
#     prevo VARCHAR(100),
#     evo VARCHAR(100),
#     evoType VARCHAR(30),
#     canGigantamax VARCHAR(100),
#     type_1 VARCHAR(30),
#     type_2 VARCHAR(30),
#     hp INT,
#     atk INT,
#     def INT,
#     spa INT,
#     spd INT,
#     spe INT,
#     ability_0 VARCHAR(100),
#     ability_1 VARCHAR(100),
#     ability_H VARCHAR(100),
#     dw_fecha_registro DATETIME,
#     dw_id_carga INT,
#     dw_deleted TINYINT(1) DEFAULT 0 NOT NULL,
#     dw_source VARCHAR(50),
#     dw_endpoint TEXT
# ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;